<a href="https://colab.research.google.com/github/SB-Chalmers/sustainable_building_website/blob/main/RG_Bibupdate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A python script to generate a bib file from swepub rss feeds
This script is meant to be run regularly and the resulting *.bib file must be used with the Hugo Academic CLI.

Run `academic import --bibtex library.bib` to update all publications in the hugo terminal.

Note: This does not work well with name structures other than firstname-lastname, to work around this there is a manual string search done at the end. Ensure that this is kept up to date.

The swepub link must also be kept up to date using the RG Publication [excel document](https://chalmers-my.sharepoint.com/:x:/g/personal/ssanjay_chalmers_se/EWB9woXUNjJPv-xcqglRiUIB_AWbEI4Ws4BX4VnpFY99Rg?e=0B4WwW)

In [1]:
!pip install switch
!pip install feedparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for switch: filename=switch-1.1.0-py3-none-any.whl size=3555 sha256=9ce9c37ab2c75fbcfcbba425464458993dccbaef24043fc6d58e3f6d62349c95
  Stored in directory: /root/.cache/pip/wheels/6e/c0/4f/f8f1f2da2f41c5ac46a26487f6c29e6ebe40d2534ac327afa8
Successfully built switch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 4.1 MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=812630b07a2c8acf0bf6a2a9ec906f048320b7d4e9e10e2b49968098bd10f4ef
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k


In [259]:
import feedparser
import argparse
import json

In [260]:
bibfeed = feedparser.parse("https://swepub.kb.se/atom.jsp?q=(%27(WFRF:(N%C3%A4geli+Claudio+1987)%20OR%20WFRF:(Jimenez+Encarnacion+Divia+1989)%20OR%20WFRF:(Galimshina+Alina)%20OR%20WFRF:(Gonzalez+Caceres+Alex+Arnoldo+1982)%20OR%20WFRF:(Holger+Wallbaum+1967)%20OR%20WFRF:(Hollberg+Alexander+1985)%20OR%20WFRF:(Lanau+Maud+1989)%20OR%20WFRF:(Rosado+Leonardo+1975)%20OR%20WFRF:(Malakhatka+Elena)%20OR%20WFRF:(Melina+Forooraghi+1989)%20OR%20WFRF:(%C3%96sterbring+Magnus+1986)%20OR%20WFRF:(Quan+Jin+1983)%20OR%20WFRF:(Sanjay+Somanath+1994)%20OR%20WFRF:(Shea+Hagy+1982)%20OR%20WFRF:(Sjouke+Beemsterboer+1984)%20OR%20WFRF:(Toivo+S%C3%A4w%C3%A9n+1993)%20OR%20WFRF:(Xinyue+Wang+1995)%20OR%20WFRF:(Yutaka+Goto+1984))%27)%27&order_by=rank2&format=RIS")

In [261]:
def add_entry(entry):
  # Key value pairs
  keys = []
  vals = []
  keys.append('PY')
  vals.append(entry.published[:4])
  for data in entry.content[0].value.split('\n'):
    keys.append(data[:2])
    vals.append(data[6:])
  # Containers for information
  author_list=[]
  entry_type = None
  title=None
  journal=None
  volume=None
  year=None
  month=None
  startingpage=None
  finalpage=None
  publisher=None
  doi=None
  abstract = None

  for i in range(len(keys)):
    case = keys[i]
    value = vals[i]
    if case == 'TY':
      entry_type = value
    elif case == 'A1' or case == 'AU':
      if value == 'Chalmers University of TechnologyChalmers tekniska högskola' or value == 'Chalmers tekniska högskolaChalmers University of Technology':
        pass
      else:
        author_list.append(value)
    elif case == 'T1' or case == 'TY' or case == 'T1':
      title=value
    elif case == 'JA' or case == 'JF':
      journal=value
    elif case == 'VL' or case == 'IS':
      volume=value
    elif case == 'PY':
      year=value.rsplit('/')[0]
    elif case == 'SP':
      startingpage=value
    elif case == 'EP':
      finalpage=value
    elif case == 'UR':
      if 'doi' in value:
        doi=value[16:].strip()
    elif case == 'PB':
      publisher=value
    elif case == 'N2':
      abstract = value
  lines=[]
  ##### JOURNALS #####
  if entry_type == 'JOUR':
    firstauthor=author_list[0].rsplit(',')[0].strip(' ')
    split = title.split(' ')
    if len(split) == 1:
      lines.append('@article{'+firstauthor.lower()+year+'_'+split[0]+',')
    else:
      lines.append('@article{'+firstauthor.lower()+year+'_'+split[0]+'_'+split[1]+',')
    authorline=' '*4 + 'author={' + ' and '.join(author_list)+'},'
    lines.append(authorline)
    if title is not None:
        lines.append(' '*4 + 'title={' + title + '},')
    if journal is not None:
        lines.append(' '*4 + 'journal={' + journal + '},')
    if volume is not None:
        lines.append(' '*4 + 'volume={' + volume + '},')
    if startingpage is not None and finalpage is not None:
        lines.append(' '*4 + 'pages={' + startingpage + '--'+finalpage+'},')
    if year is not None:
        lines.append(' '*4 + 'year={' + year + '},')
    if doi is not None:
        lines.append(' '*4 + 'doi={' + doi + '},')
    if publisher is not None:
        lines.append(' '*4 + 'publisher={' + publisher + '},')
    if abstract is not None:
        lines.append(' '*4 + 'abstract={' + abstract + '},')
  ##### BOOKS ####
  elif entry_type == 'BOOK':
    firstauthor=author_list[0].rsplit(',')[0].strip(' ')
    split = title.split(' ')
    if len(split) == 1:
      lines.append('@article{'+firstauthor.lower()+year+'_'+split[0]+',')
    else:
      lines.append('@article{'+firstauthor.lower()+year+'_'+split[0]+'_'+split[1]+',')
    authorline=' '*4 + 'author={' + ' and '.join(author_list)+'},'
    lines.append(authorline)
    if title is not None:
        lines.append(' '*4 + 'title={' + title + '},')
    if year is not None:
        lines.append(' '*4 + 'year={' + year + '},')
    if publisher is not None:
        lines.append(' '*4 + 'publisher={' + publisher + '},')
    if abstract is not None:
        lines.append(' '*4 + 'abstract={' + abstract + '},')
  lines.append('}\n')
  return lines

In [262]:
with open('library.bib','w') as f:
  for entry in bibfeed.entries:
    f.write('\n'.join(add_entry(entry)))

In [263]:
# Read in the file
with open('library.bib', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('Gonzalez Caceres, Alex Arnoldo', 'Gonzalez, Alex')
filedata = filedata.replace('Jimenez Encarnacion, Divia', 'Jimenez, Divia')
# Write the file out again
with open('library.bib', 'w') as file:
  file.write(filedata)